<a href="https://colab.research.google.com/github/takatakamanbou/ML/blob/2023/omake04.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML omake04

<img width=64 src="https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/ML-logo.png">


----
# 小さなカラー画像をニューラルネットに識別させてみよう
----

omake03 では最短距離法と最近傍法による識別の実験を行いました．ここでは，ex06notebookC のコードを再利用して，ニューラルネットワークに識別させる実験を行ってみましょう．正答率は改善されるでしょうか？

---
## 準備

#### GPUを利用するようにランタイムのタイプを変更する



ex06notebookC と同様に，GPU を利用する設定にしましょう．


1. メニューの「ランタイム」 > 「ランタイムのタイプを変更」 を選択．
1. 「ノートブックの設定」というポップアップウィンドウが開くので，「ハードウェアアクセラレータ」を「None」から「GPU」に変更し，「保存」する
1. いつもどおりコードセルを実行する．すでに実行していた場合，「以前のランタイムを削除する」というポップアップウィンドウが現れるので，「OK」を押しし，一番最初のコードセルから実行し直ます．



### いろいろ import

In [ ]:
# 準備あれこれ
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn
seaborn.set()

In [ ]:
# PyTorch 関係のほげ
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torchvision.transforms import ToTensor, Normalize, Compose
from torchvision.datasets import CIFAR10
import torchsummary

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

### データの入手



ここでは，CIFAR10 と呼ばれる，10クラスから成る小さな画像のデータセットを扱います．オリジナルの CIFAR10 は各クラス6000枚，合計6万枚の画像から成りますが，ここではそこからランダムに600x10 = 6000枚を抽出したものを使うことにします．

In [ ]:
! wget -nc https://www-tlab.math.ryukoku.ac.jp/~takataka/course/ML/minicifar10.npz
cifar10 = np.load('minicifar10.npz')
XL = cifar10['datL'].astype(float)
YL = cifar10['labL']
XT = cifar10['datT'].astype(float)
YT = cifar10['labT']
classname = cifar10['classname']
print(XL.shape, YL.shape, XT.shape, YT.shape)

`XL` が学習用のデータの配列，`YL` がそのクラスラベルを表す整数値，`XT` がテスト用のデータの配列，`YT` がそのクラスラベルを表す整数値です．クラスラベルの整数値は 0 から 9 までで，それぞれ次のものを表します．

In [ ]:
K = 10
for ik in range(K):
    print(ik, classname[ik])

### 画像を眺めてみる

画像を表示させる関数を定義しておきます．ex03notebookA や B で使っている `display` 関数と似てますが使い方が少し違うので注意．

In [ ]:
#####  データの最初の nx x ny 枚を可視化する関数（カラー版）の定義
#         （画素値の格納順は [H, W, C] で C は BGR ではなく RGB を仮定）
#
def mosaicImageC(ax, dat, nx, ny, nrow=32, ncol=32, gap=4):

    # 並べた画像の幅と高さ
    width  = nx * (ncol + gap) + gap
    height = ny * (nrow + gap) + gap

    # 画像の作成
    img = np.zeros((height, width, 3), dtype=int) + 128
    for iy in range(ny):
        lty = iy*(nrow + gap) + gap
        for ix in range(nx):
            if iy*nx+ix >= dat.shape[0]:
                break
            ltx = ix*(ncol + gap) + gap
            img[lty:lty+nrow, ltx:ltx+ncol, :] = dat[iy*nx+ix, :].reshape((nrow, ncol, 3))

    # 表示
    ax.axis('off')
    ax.imshow(img)

次のセルを実行すると，`XL` の最初の 40 個のデータを 8 列 x 5 行に並べた画像を表示します．

In [ ]:
fig, ax = plt.subplots(figsize=(8, 8))
mosaicImageC(ax, XL[:40, :], 8, 5)
plt.show()

----
## 実験1

上で用意したデータを使って実験しましょう．


#### 関数等の定義

実験で使う関数等の定義をしておきます．以下のセルに，ex06notebookC 実験1の「関数等の定義」に書いてあるコードをコピー＆ペーストしましょう（注）．対象のコードセルは4つあります．

※注意: 次のようにすると楽かも：
1. コピー元のコードセルをクリックして選択し，右クリックまたは右上の「:」から「セルをコピー」．複数セルをまとめて選択することもできます．
1. ペースト先のセルを選択して CTRL + V (Windows) または Command + V (macOS)


### 実験

次のようにして学習とテストを実行しましょう．

1. ex06notebookC 実験1の「実験」に書いてあるコードを ↓ のセルにコピー＆ペースト
1. 一部を修正する：
    - 次の行の `mnist` を `cifar10` にする．
```
dsL = MMDataset(minimnist, 'L')
dsT = MMDataset(minimnist, 'T')
```
    - 次の行の `784` を `3072` にする．MNIST のデータは $28\times 28 = 784$ 次元でしたが，CIFAR10 のデータは $32 \times 32 \times 3 = 3072$ 次元あります．
```
# ネットワークモデルの定義
neurons = [784, 10]             # ロジスティック回帰
#neurons = [784, 1000, 10]       # 2層（中間層1層）の階層型ニューラルネット
#neurons = [784, 1000, 1000, 10] # 3層（中間層2層）の階層型ニューラルネット
```
    - 次の行の `784` を `3072` にする．
```
# torchsummary
torchsummary.summary(net, (1, 784))
```
    - 条件によっては「学習曲線の表示」で描く二つのグラフの縦軸範囲が適切でなくなる場合があります．`ax[0].set_ylim()` と `ax[1].set_ylim()` の引数を変更するとよいでしょう．
1. ロジスティック回帰の実験をやって，学習データ／テストデータに対する損失および識別率の結果をメモしましょう（notebook中の適当な場所で「+テキスト」ボタンを押してセルを作り，その中に貼り付けておくとよいでしょう）．
1. 2層や3層のニューラルネットでも実験して同様に結果を残しましょう．どんな違いがあるか考えてみてね．

----
## 実験2

実験1では CIFAR10 データセットの一部（学習5000枚，テスト1000枚）しか使っていませんが，全部のデータ（学習5万枚，テスト1万枚）を使った実験をやってみましょう．

1. 実験1の「実験」のコードセルの
```
dsL = MMDataset(cifar10, 'L')
dsT = MMDataset(cifar10, 'T')
```
の箇所を以下で置き換えます．
```
transform = Compose([
    ToTensor(),
    Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])
dsL = CIFAR10(root='.', download=True, train=True, transform=transform)
dsT = CIFAR10(root='.', download=True, train=False, transform=transform)
```
1. 学習の繰り返し回数を 20 程度に減らします．学習データ数が 10 倍になるので 1epoch あたりパラメータ更新回数も 10 倍なので，epoch 数は減らしても大丈夫．
```
# 学習の繰り返し回数
nepoch = 50
```
1. ニューラルネットの構造は実験1と同じですが，結果はどのように変わるでしょうか？

## 畳み込みニューラルネットにしたい？

ex06notebookC の「よだんだよん」にも書きましたが，画像データを扱う場合，ここで使っているような「全結合型」ニューラルネットワークではなく，「畳み込み」という演算を行う層を持つ「畳み込みニューラルネット」の方が高い汎化性能が得られることが多いです．

上記のコードの場合，`MLP` クラスの定義の中身を書き換えれば畳み込みニューラルネットにすることができます．興味があれば調べてみるとよいでしょう．